# 模型摘要能力测试

初步结论：

- 即使是gpt4turbo，生成的文字效果也不令人满意
- 出现的问题主要有
    - 任务对应的人写错
    - 漏掉个别人
    - 无法按照指定的格式显示
 
改为json输出后

- 可以拿到正确的结果
- 不能保证复杂情况下也能稳定得到正确结果

完善提示词后

- 输出符合要求，没有错误
- 同样的，不能保证复杂情况下也能稳定得到正确结果

## 生成数据节点和摘要索引

In [1]:
%%time

# 模拟一个当天的全体人员工作日志
items=[
"开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引",
"开发部陈逸飞，simple-rag 产品，修正bug：中文断句不完整",
"产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询",
"产品部林晓琪，江南皮革厂客服助手项目，和客户开会，沟通需求",
"运维部李泽言，江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题",
"销售部韩雪薇，天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案",
"法务部张峻宁，完成审核了与深圳做大事公司的商务合同",
"开发部郑千，simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误",
]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [2]:
from llama_index.core.schema import TextNode

nodes=[ TextNode(text=item, id_=f"node_{index}") for index, item in enumerate(items)]
nodes

[TextNode(id_='node_0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='node_1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部陈逸飞，simple-rag 产品，修正bug：中文断句不完整', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='node_2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='no

In [3]:
%%time


from llama_index.core import SummaryIndex

summary_index = SummaryIndex(nodes)

CPU times: user 203 ms, sys: 24.5 ms, total: 228 ms
Wall time: 227 ms


## 通过提示词形成总结

In [14]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

CPU times: user 164 µs, sys: 22 µs, total: 186 µs
Wall time: 188 µs


In [15]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

retriever=summary_index.as_retriever()

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
    "注意：不要遗漏人员。"
    "注意：一个人可能有多个任务，不要遗漏任务。"
)

print(str(response))

产品部林晓琪：
1. 提交了simple-rag产品的多数据源融合检索查询新特性需求；
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误。

开发部陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 完成了江南皮革厂客服平台系统扩容，解决了负载过高导致的服务卡顿问题。

开发部郑千：
1. 修正了simple-rag产品的聊天界面markdown格式显示错误。

销售部韩雪薇：
1. 准备并完成了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案工作。

法务部张峻宁：
1. 审核并通过了与深圳做大事公司的商务合同。
CPU times: user 55.3 ms, sys: 0 ns, total: 55.3 ms
Wall time: 4.17 s


In [13]:
Settings.llm=OpenAILike(
    model="qwen-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
    "注意：不要遗漏人员。"
    "注意：一个人可能有多个任务，不要遗漏任务。"
)

print(str(response))

产品部林晓琪：
1. 提交了simple-rag产品的多数据源融合检索查询新特性需求；
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误bug。

开发部陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 参与了simple-rag产品的其他相关工作，但具体任务未详细说明。

运维部李泽言：
1. 解决了江南皮革厂客服平台的系统扩容问题，解决了负载过高导致的服务卡顿问题。

销售部韩雪薇：
1. 准备了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案工作。

法务部张峻宁：
1. 审核并完成了与深圳做大事公司的商务合同。


In [16]:
Settings.llm=OpenAILike(
    model="glm-4", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
    "注意：不要遗漏人员。"
    "注意：一个人可能有多个任务，不要遗漏任务。"
)

print(str(response))

产品部林晓琪：
1. 提交了simple-rag产品的多数据源融合检索查询新特性需求；
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误。

开发部陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 完成了江南皮革厂客服平台系统扩容，解决了负载过高导致的服务卡顿问题。

开发部郑千：
1. 修正了simple-rag产品的聊天界面markdown格式显示错误。

销售部韩雪薇：
1. 准备并完成了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案工作。

法务部张峻宁：
1. 审核并通过了与深圳做大事公司的商务合同。


In [17]:
Settings.llm=OpenAILike(
    model="gpt-3.5-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
    "注意：不要遗漏人员。"
    "注意：一个人可能有多个任务，不要遗漏任务。"
)

print(str(response))

产品部，林晓琪：
1. 提交了simple-rag产品的多数据源融合检索查询新特性需求；
2. 参与江南皮革厂客服助手项目的客户会议，沟通需求。

开发部，韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误问题。

开发部，陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 完成了simple-rag产品中聊天界面的markdown格式显示修复工作。

运维部，李泽言：
1. 解决了江南皮革厂客服平台的负载过高问题，进行了系统扩容。

销售部，韩雪薇：
1. 准备了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案撰写。

法务部，张峻宁：
1. 审核完成了与深圳做大事公司的商务合同。


In [18]:
Settings.llm=OpenAILike(
    model="gpt-4-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
    "注意：不要遗漏人员。"
    "注意：一个人可能有多个任务，不要遗漏任务。"
)

print(str(response))

产品部林晓琪：
1. 提交了simple-rag产品的多数据源融合检索查询新特性需求；
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误。

开发部陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 完成了江南皮革厂客服平台系统扩容，解决了服务卡问题。

运维部李泽言：
1. 解决了江南皮革厂客服平台的负载过高问题，进行了系统扩容。

销售部韩雪薇：
1. 准备了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案工作。

法务部张峻宁：
1. 审核完成了与深圳做大事公司的商务合同。


In [19]:
Settings.llm=OpenAILike(
    model="moonshot-v1-8k", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "按照部门、人员，总结完成的任务。"
    "比如："
    "产品部，张三，完成了如下工作：1，xxxx; 2, xxxx ..."
    "开发部，李四，完成了如下工作：1，xxxx; 2, xxxx ..."
    "       王五，完成了如下工作：1，xxxx; 2, xxxx ..."
    "注意：不要遗漏人员。"
    "注意：一个人可能有多个任务，不要遗漏任务。"
)

print(str(response))

产品部林晓琪：
1. 提交了simple-rag产品的多数据源融合检索查询新特性需求；
2. 参与了江南皮革厂客服助手项目的客户会议，沟通需求。

开发部韩宇轩：
1. 实现了simple-rag产品的文档摘要索引新特性；
2. 修正了simple-rag产品中的markdown格式显示错误。

开发部陈逸飞：
1. 修正了simple-rag产品中的中文断句不完整bug；
2. 完成了江南皮革厂客服平台系统扩容，解决了负载过高导致的服务卡顿问题。

开发部郑千：
1. 修正了simple-rag产品的聊天界面markdown格式显示错误。

销售部韩雪薇：
1. 准备并完成了天津化工集团后勤服务平台的标书商务部分；
2. 完成了公司资质方面的文案工作。

法务部张峻宁：
1. 审核并通过了与深圳做大事公司的商务合同。


In [20]:
Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "根据上下文，给出按照部门、人员、具体任务的叙述顺序的全体人员工作总结。"
)

print(str(response))

1. 开发部韩宇轩：在simple-rag产品中实现了新特性——针对文档的摘要索引。

2. 开发部陈逸飞：在simple-rag产品中修正了bug，解决了中文断句不完整的问题。

3. 产品部林晓琪：向江南皮革厂客服助手项目提交了多数据源融合检索查询的新特性需求，并与客户开会沟通需求。

4. 运维部李泽言：对江南皮革厂客服平台进行了系统扩容，成功解决因负载过高导致的服务卡顿问题。

5. 销售部韩雪薇：为天津化工集团后勤服务平台准备标书的商务部分，完成了公司资质方面的文案工作。

6. 法务部张峻宁：审核并完成了与深圳做大事公司的商务合同。

7. 开发部郑千：在simple-rag产品中修正了聊天界面markdown格式显示错误的问题。


In [22]:
response = query_engine.query(
    "开发部全体人员的工作总结"
    "列表格式，按照人员分别列出他做的事情"
)

print(str(response))

开发部韩宇轩：
- 实现了simple-rag产品的新特性：针对文档的摘要索引。

开发部陈逸飞：
- 在simple-rag产品中修正了一个bug：中文断句不完整的问题。

开发部郑千：
- 在simple-rag产品中修正了聊天界面markdown格式显示错误的问题。


In [23]:
response = query_engine.query(
    "开发部全体人员的工作总结"
    "列表格式，按照人员分别列出他做的事情"
    "格式: "
    "**开发部**"
    "- 张三:"
    "  - xxx产品修正bug，xxxx"
    "  - xxx项目新特性，xxxx"
    "- 李四:"
    "  - xxx项目修正bug，xxxx"
)

print(str(response))

**开发部**- 韩宇轩:
- simple-rag 产品：实现新特性——针对文档的摘要索引

**开发部**- 陈逸飞:
- simple-rag 产品：修正bug——中文断句不完整

**开发部**- 郑千:
- simple-rag 产品：修正bug——聊天界面 markdown 格式显示错误


In [24]:
Settings.llm=OpenAILike(
    model="gpt-4-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

response = query_engine.query(
    "开发部全体人员的工作总结"
    "列表格式，按照人员分别列出他做的事情"
    "格式: "
    "**开发部**"
    "- 张三:"
    "  - xxx产品修正bug，xxxx"
    "  - xxx项目新特性，xxxx"
    "- 李四:"
    "  - xxx项目修正bug，xxxx"
)

print(str(response))

**开发部**- 韩宇轩:
- simple-rag 产品：实现新特性——针对文档的摘要索引

**开发部**- 陈逸飞:
- simple-rag 产品：修正 bug —— 中文断句不完整

**开发部**- 郑千:
- simple-rag 产品：修正 bug —— 聊天界面 markdown 格式显示错误


In [25]:
Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

response = query_engine.query(
    "给出涉及的所有人，以及他们的部门"
)

print(str(response))

涉及的人员及其所属部门包括：

1. 韩宇轩 - 开发部
2. 陈逸飞 - 开发部
3. 林晓琪 - 产品部
4. 李泽言 - 运维部
5. 韩雪薇 - 销售部
6. 张峻宁 - 法务部
7. 郑千 - 开发部


In [26]:
response = query_engine.query(
    "给出涉及的部门"
)

print(str(response))

涉及的部门包括：开发部、产品部、运维部、销售部和法务部。


In [27]:
response = query_engine.query(
    "给出涉及的产品或者项目"
)

print(str(response))

涉及的产品或项目包括：
1. simple-rag 产品 - 包含新特性的实现、bug修正和需求提交。
2. 江南皮革厂客服助手项目 - 需求沟通与问题解决。
3. 天津化工集团后勤服务平台 - 标书准备及商务部分完成。
4. 深圳做大事公司的商务合同 - 法务部的审核工作。


In [33]:
response = query_engine.query(
    "只显示上下文涉及的部门"
    "列出上下文涉及的部门，按照下面的顺序"
    "部门输出的顺序按照： 市场部、财务部、销售部、产品部、开发部、运维部、法务部、后系统"
)

print(str(response))

市场部、销售部、产品部、开发部、运维部、法务部


In [36]:
response = query_engine.query(
    "列出只涉及simple-rag产品的部门和人，以及他们做的事情"
    "列表显示"
)

print(str(response))

开发部韩宇轩：
- 实现新特性：针对文档的摘要索引。

开发部陈逸飞：
- 修正bug：中文断句不完整。

开发部郑千：
- 修正bug：聊天界面markdown格式显示错误。


## 通过提示词生成结构化数据 - json

In [37]:
Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

response = query_engine.query(
    "给出涉及的所有部门的json格式，部门列表，比如："
    "`['销售部','开发部', ..]`"
)

print(str(response))

["销售部", "开发部", "产品部", "运维部", "法务部"]


In [49]:
response = query_engine.query(
    "给出涉及的所有部门和人员的json格式，结果列表，比如："
    """
    [
        {
            "dept": "产品部",
            "persons": [
                {
                    "name": "张三",
                    "items": [
                        "xx项目，和客户开会: xxxx",
                        "xx产品，提交新特性: xxx",
                        ...
                    ]
                },
                {
                    "name": "李四",
                    "items": [
                        "xx项目，实现新特性: xxxx",
                        ...
                    ]
                },
                {
                    "name": "xxx",
                    "items": [
                        "xx项目，实现新特性: xxxx",
                        ...
                    ]
                },
                {
                    "name": "xxx",
                    "items": [
                        "xx项目，实现新特性: xxxx",
                        ...
                    ]
                },
                ...
            ]
        },
        {
            "dept": "开发部",
            "persons": [
                {
                    "name": "王五",
                    "items": [
                        "xx项目，修正bug: xxxx",
                        "xx产品，实现新特性: xxx",
                        ...
                    ]
                },
                ...
            ]
        },
        ...
    ]
    """
)

print(str(response))

[
    {
        "dept": "产品部",
        "persons": [
            {
                "name": "林晓琪",
                "items": [
                    "simple-rag 产品，提交新特性需求：多数据源的融合检索查询",
                    "江南皮革厂客服助手项目，和客户开会，沟通需求"
                ]
            }
        ]
    },
    {
        "dept": "开发部",
        "persons": [
            {
                "name": "韩宇轩",
                "items": [
                    "simple-rag 产品，实现新特性：针对文档的摘要索引"
                ]
            },
            {
                "name": "陈逸飞",
                "items": [
                    "simple-rag 产品，修正bug：中文断句不完整"
                ]
            },
            {
                "name": "郑千",
                "items": [
                    "simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误"
                ]
            }
        ]
    },
    {
        "dept": "运维部",
        "persons": [
            {
                "name": "李泽言",
                "items": [
                    "江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题"
     

## 完善提示词的对话输出

In [58]:
%%time

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

response = query_engine.query(
    "给出涉及的所有部门的输出，输出格式见："
    """
    产品部：

    - 张三
        - xxx产品，提交新特性需求: xxx
        - xxx项目，和客户开会，xxxxx
        - ...
    - 李四
        - xxxx
        - ...
    - 王五
      - ...

    xx部：
    
    - 赵六
      - ...

    xxx部:
    ...
    """
)

print(str(response))

产品部：

- 林晓琪
    - simple-rag 产品，提交新特性需求：多数据源的融合检索查询
    - 江南皮革厂客服助手项目，和客户开会，沟通需求

开发部：

- 韩宇轩
    - simple-rag 产品，实现新特性：针对文档的摘要索引
- 陈逸飞
    - simple-rag 产品，修正bug：中文断句不完整
- 郑千
    - simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误

运维部：

- 李泽言
    - 江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题

销售部：

- 韩雪薇
    - 天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案

法务部：

- 张峻宁
    - 完成审核了与深圳做大事公司的商务合同
CPU times: user 9.1 ms, sys: 116 µs, total: 9.21 ms
Wall time: 4.33 s
